In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys

In [2]:
# Load dataset
df = pd.read_csv("dataset/resize_borvo.csv")

# Konversi ke grid dictionary
grid = {}
for _, row in df.iterrows():
    grid[(int(row["i"]), int(row["j"]))] = {"H": row["H"], "W": row["W"]}

In [3]:
# Fungsi ambil tetangga 8 arah (termasuk diagonal)
def get_all_neighbors(pos):
    i, j = pos
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0), (1, 1)
    ]
    neighbors = [(i + di, j + dj) for di, dj in directions]
    return [n for n in neighbors if n in grid]

# Fungsi hitung skor W/E
def get_score(current_pos, neighbor_pos):
    H1 = grid[current_pos]["H"]
    H2 = grid[neighbor_pos]["H"]
    S = H1 - H2

    if S > 1.5:
        return -np.inf  # Tidak aman → skip
    
    E = 1 + 1 * abs(S)
    W = grid[neighbor_pos]["W"]
    return W / E

In [4]:
def hill_climbing_with_horizon(start, depth=3):
    if start not in grid:
        print(f"Titik {start} tidak ditemukan dalam dataset.")
        return [], [], [] # Kembalikan 3 list kosong

    current = start
    path = [current]
    visited = set([start])

    # Siapkan list untuk menyimpan data performa dan energi
    decision_scores = []
    energy_consumption = [] # <-- LIST BARU UNTUK ENERGI

    def dfs(current_path, total_score, remaining_depth):
        current_node = current_path[-1]
        if remaining_depth == 0:
            return (total_score, current_path)
        best = (total_score, current_path)
        for neighbor in get_all_neighbors(current_node):
            if neighbor in current_path:
                continue
            score = get_score(current_node, neighbor)
            if score == -np.inf:
                continue
            result = dfs(current_path + [neighbor], total_score + score, remaining_depth - 1)
            if result[0] > best[0]:
                best = result
        return best

    while True:
        best_score_for_move = -np.inf
        best_path_ahead = None
        best_single_step_score = 0
        best_energy_for_step = 0 # <-- Variabel untuk simpan energi langkah terbaik
        neighbors = get_all_neighbors(current)
        for neighbor in neighbors:
            if neighbor in visited:
                continue
            
            single_step_score = get_score(current, neighbor)
            if single_step_score == -np.inf:
                continue
            
            # Hitung energi untuk langkah ini
            H1 = grid[current]["H"]
            H2 = grid[neighbor]["H"]
            S = H1 - H2
            energy_for_this_step = 1 + abs(S)

            cumulative_score, result_path = dfs([current, neighbor], single_step_score, depth - 1)

            if cumulative_score > best_score_for_move:
                best_score_for_move = cumulative_score
                best_path_ahead = result_path
                best_single_step_score = single_step_score
                best_energy_for_step = energy_for_this_step # <-- Simpan energi

        if best_path_ahead is None or best_path_ahead[1] in visited:
            break

        current = best_path_ahead[1]
        path.append(current)
        visited.add(current)
        
        decision_scores.append(best_single_step_score)
        energy_consumption.append(best_energy_for_step) # <-- Tambahkan energi ke list

    final_scores_for_plotting = [0] + decision_scores

    # KEMBALIKAN 3 NILAI SEKARANG
    return path, final_scores_for_plotting, energy_consumption

In [5]:
# Titik uji coba
test_points = [
    (540/2, 648/2),
    (20/2, 502/2),
    (410//2, 915//2),
    (529, 357)
]

# Jalankan algoritma untuk semua titik uji
for idx, point in enumerate(test_points, 1):
    print(f"\n=== Titik {idx} - Start di {point} ===")
    path = hill_climbing_with_horizon(point)
    print(f"Jumlah langkah: {len(path)}")
    print("Jalur:")
    for p in path:
        print(p)


=== Titik 1 - Start di (270.0, 324.0) ===
Jumlah langkah: 3
Jalur:
[(270.0, 324.0), (270.0, 323.0), (269.0, 322.0), (270.0, 321.0), (271.0, 320.0), (270.0, 319.0), (269.0, 319.0), (269.0, 320.0), (269.0, 321.0), (270.0, 322.0), (269.0, 323.0), (269.0, 324.0), (268.0, 324.0), (268.0, 325.0), (267.0, 326.0), (266.0, 326.0), (266.0, 325.0), (267.0, 325.0), (267.0, 324.0), (267.0, 323.0), (268.0, 322.0), (267.0, 321.0), (266.0, 321.0), (266.0, 320.0), (265.0, 321.0), (264.0, 321.0), (263.0, 322.0), (262.0, 321.0), (262.0, 322.0), (263.0, 321.0), (264.0, 320.0), (265.0, 319.0), (266.0, 318.0), (267.0, 318.0), (268.0, 317.0), (267.0, 316.0), (266.0, 317.0), (265.0, 317.0), (264.0, 317.0), (263.0, 317.0), (263.0, 316.0), (263.0, 315.0), (264.0, 316.0), (265.0, 315.0), (265.0, 314.0), (266.0, 315.0), (267.0, 315.0), (266.0, 314.0), (265.0, 313.0), (266.0, 313.0), (265.0, 312.0), (264.0, 312.0), (265.0, 311.0), (265.0, 310.0), (264.0, 309.0), (264.0, 310.0), (263.0, 309.0), (264.0, 308.0), (26

In [ ]:
# =====================================================================
# --- BAGIAN 1: KONFIGURASI SIMULASI ---
# =====================================================================
# Daftar titik awal untuk disimulasikan
test_points = [
    (270, 324),
    (10, 251),
    (410//2, 915//2)
]
nama_file_peta = 'europa_map/raw/resize_borvo_mensa.jpg' # Pastikan nama peta sudah benar

# Loop untuk menjalankan simulasi untuk setiap titik awal
for idx, titik_awal in enumerate(test_points, 1):
    nama_file_output = f'simulasi_HC_{idx}_start_at_{titik_awal}.gif'

    print("\n" + "="*60)
    print(f"--- MEMULAI SIMULASI {idx} DARI {len(test_points)} | START: {titik_awal} ---")
    print("="*60)

    # =====================================================================
    # --- BAGIAN 2: PERSIAPAN DATA ---
    # =====================================================================
    print("1. Menjalankan algoritma Hill Climbing...")
    try:
        # PERHATIKAN: Sekarang kita menerima 3 variabel
        jalur, data_performa, energi_per_langkah = hill_climbing_with_horizon(titik_awal, depth=5)
    except Exception as e:
        print(f"!!! TERJADI ERROR SAAT MENJALANKAN ALGORITMA: {e}")
        continue # Lanjut ke titik berikutnya jika error

    if not jalur or len(jalur) < 2:
        print(f"!!! ALGORITMA HANYA MENGHASILKAN {len(jalur)} TITIK. Melanjutkan ke titik berikutnya.")
        continue

    # HITUNG TOTAL ENERGI
    total_energi = sum(energi_per_langkah)

    print(f"2. Algoritma selesai. Ditemukan jalur dengan {len(jalur)} langkah.")
    print(f"   >> Total Konsumsi Energi: {total_energi:.2f}") # Tampilkan di terminal
    langkah_total = range(len(jalur))

    # =====================================================================
    # --- BAGIAN 3: PERSIAPAN PLOT ---
    # =====================================================================
    print("3. Menyiapkan 'panggung' untuk rendering animasi...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    fig.suptitle(f'Simulasi Robot HC ke {idx} di Europa (Start: {titik_awal})', fontsize=16)

    try:
        img = plt.imread(nama_file_peta)
        ax1.imshow(img)
    except FileNotFoundError:
        ax1.set_facecolor('black')

    jejak_robot, = ax1.plot([], [], 'b-', lw=2, alpha=0.7, label='Jejak Dilalui', zorder=5)
    posisi_robot, = ax1.plot([], [], 'ro', ms=8, label='Posisi Robot', zorder=10) 
    ax1.legend()

    ax2.set_title('Grafik Performa (Nilai Heuristik)')
    ax2.set_xlabel('Langkah')
    ax2.set_ylabel('Skor Keputusan (W/E)')
    ax2.set_xlim(0, len(jalur))
    if data_performa:
        ax2.set_ylim(min(data_performa) - 0.5, max(data_performa) + 0.5) # Perbaiki rentang Y
    ax2.grid(True)
    grafik_performa, = ax2.plot([], [], 'g-', marker='o', ms=4)
    
    # TAMBAHKAN TEKS ENERGI PADA GRAFIK
    ax2.text(0.95, 0.95, f'Total Energi: {total_energi:.2f}',
             transform=ax2.transAxes, fontsize=12,
             verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
             
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # =====================================================================
    # --- BAGIAN 4: FUNGSI UPDATE (TIDAK BERUBAH) ---
    # =====================================================================
    def update(frame):
        if frame % 20 == 0:
            print(f"   ...merender frame {frame}/{len(jalur)}")
        
        path_sejauh_ini = jalur[:frame+1]
        koordinat_j = [titik[1] for titik in path_sejauh_ini]
        koordinat_i = [titik[0] for titik in path_sejauh_ini]
        
        jejak_robot.set_data(koordinat_j, koordinat_i)
        posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])
        grafik_performa.set_data(langkah_total[:frame+1], data_performa[:frame+1])
        ax1.set_title(f'Peta Penjelajahan (Langkah: {frame})')
        return jejak_robot, posisi_robot, grafik_performa,

    # =====================================================================
    # --- BAGIAN 5: RENDER DAN SIMPAN KE FILE ---
    # =====================================================================
    print("4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...")
    ani = animation.FuncAnimation(fig, update, frames=len(jalur), blit=False)

    try:
        ani.save(nama_file_output, writer='pillow', fps=20)
        
        print("\n" + "#"*60)
        print(f"###   BERHASIL! Animasi telah disimpan sebagai '{nama_file_output}'   ###")
        print("#"*60)

    except Exception as e:
        print(f"\n!!! GAGAL MENYIMPAN ANIMASI: {e}")
    
    # Tutup plot setelah selesai agar tidak menumpuk di memori
    plt.close(fig)

print("\nSEMUA SIMULASI SELESAI.")


--- MEMULAI SIMULASI 1 DARI 3 | START: (270, 324) ---
1. Menjalankan algoritma Hill Climbing...
2. Algoritma selesai. Ditemukan jalur dengan 369 langkah.
   >> Total Konsumsi Energi: 415.08
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...
   ...merender frame 0/369
   ...merender frame 0/369


C:\Users\Rhama\AppData\Local\Temp\ipykernel_20296\3895028617.py:88: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/369
   ...merender frame 40/369
   ...merender frame 60/369
   ...merender frame 80/369
   ...merender frame 100/369
   ...merender frame 120/369
   ...merender frame 140/369
   ...merender frame 160/369
   ...merender frame 180/369
   ...merender frame 200/369
   ...merender frame 220/369
   ...merender frame 240/369
   ...merender frame 260/369
   ...merender frame 280/369
   ...merender frame 300/369
   ...merender frame 320/369
   ...merender frame 340/369
   ...merender frame 360/369

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_HC_1_start_at_(270, 324).gif'   ###
############################################################

--- MEMULAI SIMULASI 2 DARI 3 | START: (10, 251) ---
1. Menjalankan algoritma Hill Climbing...
2. Algoritma selesai. Ditemukan jalur dengan 129 langkah.
   >> Total Konsumsi Energi: 145.53
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses render

C:\Users\Rhama\AppData\Local\Temp\ipykernel_20296\3895028617.py:88: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/129
   ...merender frame 40/129
   ...merender frame 60/129
   ...merender frame 80/129
   ...merender frame 100/129
   ...merender frame 120/129

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_HC_2_start_at_(10, 251).gif'   ###
############################################################

--- MEMULAI SIMULASI 3 DARI 3 | START: (205, 457) ---
1. Menjalankan algoritma Hill Climbing...
2. Algoritma selesai. Ditemukan jalur dengan 110 langkah.
   >> Total Konsumsi Energi: 114.42
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...
   ...merender frame 0/110
   ...merender frame 0/110


C:\Users\Rhama\AppData\Local\Temp\ipykernel_20296\3895028617.py:88: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/110
   ...merender frame 40/110
   ...merender frame 60/110
   ...merender frame 80/110
   ...merender frame 100/110

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_HC_3_start_at_(205, 457).gif'   ###
############################################################

SEMUA SIMULASI SELESAI.
